In [105]:
import os
import fileinput as fi
import glob
import moviepy.editor as mp

os.chdir(path=r"D:\02_batch_2")
print("Current Working Directory " , os.getcwd())


Current Working Directory  D:\02_batch_2


In [126]:
df_dirname = []
for folder in sorted(os.listdir(os.getcwd())):
    df_dirname.append((folder))

df_dirname = pd.DataFrame(df_dirname, columns=['Folder'])
df_dirname.sort_values(by='Folder',inplace=True)
df_dirname

,Folder
0,37_degrees_series
1,Almajd_kids
2,Ba3d_altes3een_Podcast
3,C4N
4,Dupamicaffeine
5,Mojtm3_Ethraa
6,Naif_aljaweni
7,Sando9_aswad_turki_alfaisal
8,Shabab_Ela3mal
9,Update_NewMedia_Podcast


### Arabic Annotation collection

In [135]:
import pandas as pd
import glob

li = []

#df_dirname = ['37_degrees_series','abdullah_Alabdli']

for i,dname in enumerate(df_dirname['Folder']):
    path = r'D:/02_batch_2/'
    files = glob.glob(path + dname + '/*.txt')
    print(dname)
    for i,f in enumerate(files):
        stock = os.path.basename(f)

        temp_df = pd.read_csv(f, delimiter="\t")

        temp_df['filename'] = stock
        temp_df['filepath'] = path + dname + '/'
        temp_df['duration'] = temp_df.stop-temp_df.start

        temp_df['filename'] = temp_df['filename'].replace('.txt', '.wav', regex=True)

        li.append(temp_df)

        print(f'Successfully created dataframe for {dname} {stock} {nextFileName} with shape {temp_df.shape}')

df = pd.concat(li, axis=0)
print(df.shape)
df.head()

37_degrees_series
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-0.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (49, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-1.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (24, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-2.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (51, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-3.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (44, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-4.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (57, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-5.txt Folder    zainab_almatroodi
Name: 59, dtype: object with shape (27, 8)
Successfully created dataframe for 37_degrees_series 37_degrees_series_0-6.txt Fol

In [94]:
df.groupby('filepath').agg('sum')

,start,stop,duration
filepath,,,
D:/02_batch_2/37_degrees_series/,1.100569e+06,1.126571e+06,26002.161790
D:/02_batch_2/abdullah_Alabdli/,1.065524e+05,1.159934e+05,9441.028707


### Video Clipping

In [78]:
temp = df.iloc[0,5]
my_clip = mp.AudioFileClip(os.path.join(path, temp))
clip = my_clip.subclip(df.iloc[0,2],df.iloc[0,3])
clip.write_audiofile(r"D:\nf\audio\temp.mp3")



MoviePy - Writing audio in D:\nf\audio\temp.mp3


MoviePy - Done.


#### Directory Structure

In [ ]:
def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))



In [ ]:
list_files(r"D:\02_batch_2")